<a href="https://colab.research.google.com/github/Lasitha-Jayawardana/IDS/blob/main/EM_Kmeans_NSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd
import itertools
from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import math
from sklearn.cluster import KMeans
# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
from sklearn.model_selection import GridSearchCV, cross_val_score,train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix,f1_score
from sklearn import preprocessing 
from sklearn.mixture import GaussianMixture
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [20]:
def loadData():
  global df_train
  global df_test

  df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IDS Project/NSL_Train.csv')
  df_test =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IDS Project/NSL_Test.csv')

  for col in ['protocol_type', 'service', 'flag', 'class']:
    df_train[col] = df_train[col].astype('category')
    df_test[col] = df_test[col].astype('category')

  df_train=df_train.drop(columns='id')
  df_test=df_test.drop(columns='id')
  df_train['class'] = df_train['class'].map( {'normal':0, 'anomaly':1} )
  df_test['class'] = df_test['class'].map( {'normal':0, 'anomaly':1} )


In [21]:
def balanceClass():
  global df_train
  global df_test
  # Class count
  count_class_0, count_class_1 = df_train['class'].value_counts()

  # Divide by class
  df_class_1= df_train[df_train['class'] == 0]
  df_class_0 = df_train[df_train['class'] == 1]
      
  df_class_0_under = df_class_0.sample(count_class_1)
  df_train = pd.concat([df_class_0_under, df_class_1], axis=0)
  df_train.reset_index(drop=True,inplace=True)
  df_train.groupby('class')['class'].count()

In [22]:
def addOtherLabel():
  global df_train
  global df_test

  otherlabel = 'Other_service'
  others1 = df_train['service'].value_counts().index[30:]
  # apply new category label
  df_train['service'] = df_train['service'].cat.add_categories([otherlabel])
  df_train['service'] = df_train['service'].replace(others1, otherlabel)

  df_train['service'].replace({'pop_3':'Other_service','sunrpc':'Other_service',
                              'link':'Other_service','name':'Other_service','echo':'Other_service',
                              'netbios_ns':'Other_service'}, inplace = True)

  others1 = df_test['service'].value_counts().index[30:]
  # apply new category label
  df_test['service'] = df_test['service'].cat.add_categories([otherlabel])
  df_test['service'] = df_test['service'].replace(others1, otherlabel)

  df_test['service'].replace({'pop_3':'Other_service','sunrpc':'Other_service',
                              'link':'Other_service','name':'Other_service','echo':'Other_service',
                              'netbios_ns':'Other_service'}, inplace = True)



In [23]:
def removeDuplicate():
  print('Duplicates train:',df_train.duplicated().sum())
  df_train.drop_duplicates(keep='first',inplace=True)
  print(df_train.duplicated().sum())

  print('Duplicates test:',df_test.duplicated().sum())
  df_test.drop_duplicates(keep='first',inplace=True)
  print(df_test.duplicated().sum())

  df_train.reset_index(drop=True,inplace=True)
  df_test.reset_index(drop=True,inplace=True)

In [24]:
def splitData():
  global X_train
  global Y_train
  global X_test
  global Y_test
  global df_train
  global df_test
  
  limit1 = df_train.shape[1]-1
      
  X_train = df_train.iloc[:,0:limit1] # train set features
  Y_train = df_train.iloc[:,limit1]
      
  limit2 = df_test.shape[1]-1
      
  X_test = df_test.iloc[:,0:limit2] # test set features
  Y_test = df_test.iloc[:,limit2]

In [25]:
def labelEncoding(X_train,X_test):
  label_encoder = preprocessing.LabelEncoder() 
 
# Encode labels in column 'species'. 
  X_train['protocol_type']= label_encoder.fit_transform(X_train['protocol_type' ]) 
  X_test['protocol_type']= label_encoder.transform(X_test['protocol_type' ]) 
  X_train['flag']= label_encoder.fit_transform(X_train['flag' ]) 
  X_test['flag']= label_encoder.transform(X_test['flag' ]) 
  X_train['service']= label_encoder.fit_transform(X_train['service' ]) 
  X_test['service']= label_encoder.transform(X_test['service' ]) 
  return X_train,X_test

In [26]:
def one_hot_encode(X_train,X_test):# Making categorical variables into numeric representation by one- hot encoding
  
  categorical_cols = ['protocol_type','flag','service']

  # Training dataset one hot encoding
  ohe = OneHotEncoder(handle_unknown = 'ignore')
  ohe.fit(X_train[categorical_cols])
  array_hot_encoded1 = ohe.transform(X_train[categorical_cols]).toarray()

  data_hot_encoded1 = pd.DataFrame(array_hot_encoded1, index=X_train.index,columns=ohe.get_feature_names(categorical_cols))
  X_train = X_train.drop(columns=categorical_cols)
  X_train = pd.concat([data_hot_encoded1,X_train], axis=1)
  
  print('X_train shape :',X_train.shape)
  
  # Test dataset one hot encoding
  array_hot_encoded2 = ohe.transform(X_test[categorical_cols]).toarray()
  data_hot_encoded2 = pd.DataFrame(array_hot_encoded2, index=X_test.index,columns=ohe.get_feature_names(categorical_cols))
  X_test = X_test.drop(columns=categorical_cols)
  X_test = pd.concat([data_hot_encoded2,X_test], axis=1)
  
  print('X_test shape :',X_test.shape)
  
  return X_train, X_test

In [27]:
def scaling(X_train,X_test,scaler):
  
  X_train = pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)
  X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns)
  return X_train,X_test



In [28]:
def runPCA(X_train,X_test,kPCA):
  pca = PCA(n_components=kPCA)
  X_rtrain = pca.fit(X_train).transform(X_train)
  X_rtest  = pca.transform(X_test)
  return X_rtrain,X_rtest

In [29]:
def Gmm(X):
  #check 1->4 components
  tuned_parameters = {'n_components': np.array([1,2,3,4])}#,5,6,7,8,9,10
  #construct grid search object that uses 2 fold cross validation
  cv= [(slice(None), slice(None))]# for non crossvalidation
  clf = GridSearchCV(GaussianMixture(),tuned_parameters,verbose=3,cv=cv,return_train_score=True,n_jobs=1)
  #fit the data
  clf.fit(X)
  c = pd.DataFrame(clf.cv_results_)
  print(c)
  #plot the number of Gaussians against their rank
  plt.scatter(clf.cv_results_['param_n_components'],\
              clf.cv_results_['rank_test_score'])

In [30]:

def searchGmm(X):
  lowest_bic = np.infty
  bic = []
  n_components_range = range(3, 12)
  cv_types = ['spherical', 'tied', 'diag', 'full']
  for cv_type in cv_types:
      for n_components in n_components_range:
          # Fit a Gaussian mixture with EM
          print ("K = ",n_components)
          gmm = GaussianMixture(n_components=n_components,covariance_type=cv_type,init_params='kmeans',random_state=10002,verbose=0,verbose_interval=10,max_iter=1000)

          gmm.fit(X)
          bic.append(gmm.aic(X))
          if bic[-1] < lowest_bic:
              lowest_bic = bic[-1]
              best_gmm = gmm

  bic = np.array(bic)
  color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue',
                                'darkorange'])
  clf = best_gmm
  bars = []
  print(lowest_bic)
  # Plot the BIC scores
  plt.figure(figsize=(8, 6))
  spl = plt.subplot(2, 1, 1)
  for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
      xpos = np.array(n_components_range) + .2 * (i - 2)
      bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                    (i + 1) * len(n_components_range)],
                          width=.2, color=color))
  plt.xticks(n_components_range)
  plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
  plt.title('BIC score per model')
  xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
      .2 * np.floor(bic.argmin() / len(n_components_range))
  plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
  spl.set_xlabel('Number of components')
  spl.legend([b[0] for b in bars], cv_types)


In [31]:
def XtoN():
  global X_trainN
  global X_testN
  global kmeans
  X_trainN = X_train
  X_testN = X_test
  

In [32]:
# Running K means with multible seeds
 
def searchSeed(X_train,no_of_clusters):
  global kmeans
  best_seed = None


  min_inertia=0.0
  
  seeds = [0,9500,10000,10500,11000,15000,18000 , 20000, 40000, 60000, 80000,90000, 120000]
  for seed in seeds:

    
    #print('\n seed= {} \n'.format(seed))
        
    kmeans = KMeans(n_clusters=no_of_clusters, init='k-means++',random_state=seed)
    kmeans = kmeans.fit(X_train)
    
    inertia = kmeans.inertia_
    #print("The innertia for : 2 Clusters is:", inertia) 
    # if current measurement of heterogeneity is lower than previously seen,
    # update the minimum record of heterogeneity.
    if min_inertia == 0 :
        min_inertia = inertia
        best_seed = seed
    if inertia < min_inertia:
        min_inertia = inertia
        best_seed = seed
        

  print("\nMin inertia : ", min_inertia)
  print("Best Seed : ",best_seed)
  return best_seed
 

In [33]:
def runKmeans(k,best_seed):
  global kmeans
  # Running K means on K clusters
  kmeans = KMeans(n_clusters=k,init='k-means++', random_state=best_seed)
  kmeans = kmeans.fit(X_trainN)

In [34]:
def binaryaccuracyScore():
  label0 = 1- kmeans.labels_
  label1 = kmeans.labels_
  test_label1 = kmeans.predict(X_testN)
  test_label0 = 1 - test_label1

  a = accuracy_score(Y_train,label0)*100
  aa = accuracy_score(Y_train,label1)*100
  if (a>= aa):
    l = test_label0
    print("Accuracy Train: ",a)
  else:
    l = test_label1
    print("Accuracy Train: ",aa)

  print("Accuracy Test: ",accuracy_score(Y_test,l)*100)

  print("\nTest set precision : {:.4f}".format(precision_score(Y_test, l)))
  print("Test set recall    : {:.4f}".format(recall_score(Y_test, l)))
  print("Test set F1-score  : {:.4f}".format(f1_score(Y_test, l)))
  
  cm = confusion_matrix(Y_test,l,normalize= 'true')
  print('True Positives  : {:.2f}'.format(cm[1][1]))
  print('True Negatives  : {:.2f}'.format(cm[0][0]))
  print('False Positives : {:.2f}'.format(cm[0][1]))
  print('False Negatives : {:.2f}'.format(cm[1][0]))

In [44]:
def oneVsAllScore(n,Trainlabel,Testlabel): 

  def oneVsAll(number):
    if (number ==nn):
      return 0
    else:
      return 1
  m=[]
  
  for i in range(0, n):
    nn=i
    m.append(accuracy_score(Y_train,np.array(list(map(oneVsAll, Trainlabel)))))
    print(i,"  ",accuracy_score(Y_train,np.array(list(map(oneVsAll, Trainlabel)))))

  print("Accuracy Train: ",max(m)*100)
  index =m.index(max(m))
  nn=index
  #print(nn)
  a=np.array(list(map(oneVsAll, Testlabel)))
  print("Accuracy Test: ",accuracy_score(Y_test,a)*100)
  print("Test set precision : {:.4f}".format(precision_score(Y_test, a)))
  print("Test set recall    : {:.4f}".format(recall_score(Y_test, a)))
  print("Test set F1-score  : {:.4f}".format(f1_score(Y_test, a)))
  print("Test set Roc       : {:.4f}".format(roc_auc_score(Y_test, a)))
  cm = confusion_matrix(Y_test,a,normalize= 'true')
  print('True Positives  : {:.2f}'.format(cm[1][1]))
  print('True Negatives  : {:.2f}'.format(cm[0][0]))
  print('False Positives : {:.2f}'.format(cm[0][1]))
  print('False Negatives : {:.2f}'.format(cm[1][0]))
 

In [36]:
def mapMethodScore(n):
  
  l0=[]
  Trainlabel = kmeans.labels_
  Testlabel = kmeans.predict(X_testN)
  def lmap(num):
    if (num in l0):
      return 0
    else:
      return 1
  
  for i in range(0, n):

    #print("Cluster ", i)
    c = Y_train[Trainlabel==i]
    if (len(c[c == 1])<len(c[c == 0])):
      l0.append(i)
      #print("add to normal ",i)
    #print("intrusion : ",len(c[c == 1]))
    #print("normal : ",len(c[c == 0]))

  ltrain = np.array(list(map(lmap , Trainlabel)))
  print("Accuracy Train: ",accuracy_score(Y_train,ltrain)*100)

  ltest = np.array(list(map(lmap , Testlabel)))

  print("Accuracy Test: ",accuracy_score(Y_test,ltest)*100)
  print("Test set precision : {:.4f}".format(precision_score(Y_test,ltest)))
  print("Test set recall    : {:.4f}".format(recall_score(Y_test,ltest)))
  print("Test set F1-score  : {:.4f}".format(f1_score(Y_test,ltest)))
  cm = confusion_matrix(Y_test,ltest,normalize= 'true')
  print('True Positives  : {:.2f}'.format(cm[1][1]))
  print('True Negatives  : {:.2f}'.format(cm[0][0]))
  print('False Positives : {:.2f}'.format(cm[0][1]))
  print('False Negatives : {:.2f}'.format(cm[1][0]))



In [ ]:
#.......................................................................Kmeans.............................................
trainScore = True
cvCount = 5

last=1 #test=0
i_index=-1
myScore=[]
iCount=0
fitNum=0


def acc_score(actual,prediction,method):
  global last
  global i_index
  global myScore
  global iCount
  global fitNum
 

  m=[[],[],[],[],[]]
  cls = -1
  
  
  def oneVsAll(num):
    if (num ==cls):
      return 0
    else:
      return 1

  if (last == 0 and trainScore):
     #trainset
    print("in train")
    
    myScore.clear()
    cls = i_index
    #print("Index--------",i_index)
    oneVtest =np.array(list(map(oneVsAll, prediction)))
    myScore.append(accuracy_score(actual,oneVtest)*100)
    myScore.append(precision_score(actual,oneVtest))
    myScore.append(recall_score(actual,oneVtest))
    myScore.append(f1_score(actual,oneVtest))
    myScore.append(roc_auc_score(actual,oneVtest))

    last = 1
    
  else:
   #testset
    print("in test")
    lena = len(np.unique(actual))
    lenp = len(np.unique(prediction))
    if (lena>lenp):
      n = lena
    else:
      n= lenp
    if (iCount%cvCount == 0):
      
      fitNum =fitNum + 1
      print("-----------------------------------------------------------",fitNum,"--------------------------------------------------------")
      
       
    
    myScore.clear()
    
    for i in range(0, n):
      cls=i
      oneVall=np.array(list(map(oneVsAll, prediction)))
      
      m[0].append(accuracy_score(actual,oneVall)*100)
      m[1].append(precision_score(actual,oneVall))
      m[2].append(recall_score(actual,oneVall))
      m[3].append(f1_score(actual,oneVall))
      m[4].append(roc_auc_score(actual,oneVall))
      
    
    i_index = m[0].index(max(m[0]))  
    #print("Index--------",i_index)
    myScore.append(m[0][i_index])
    myScore.append(m[1][i_index])
    myScore.append(m[2][i_index])
    myScore.append(m[3][i_index])
    myScore.append(m[4][i_index])

    last = 0
    #print("accurcy max---",myScore[0])
  return myScore[0]
  


def prec_score(estimator,x,y):
  return myScore[1]

def rec_score(estimator,x,y):
  return myScore[2]

def f1_s_score(estimator,x,y):
  return myScore[3]

def roc_score(estimator,x,y):
  return myScore[4]




loadData()
#print(df_train.groupby('label')['label'].count())
#addOtherLabel()
removeDuplicate()
#balanceClass()
splitData()

X_train, X_test = one_hot_encode(X_train,X_test)



scalers_to_test = [StandardScaler()]#
 
params = {'scaler': scalers_to_test,
          'reduce_dim': [None,PCA(n_components=2),PCA(n_components=3),PCA(n_components=4)],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,60000]}
          
"""
....................................or (both result same in reduce__dim)
params = [{'scaler': scalers_to_test,
          'reduce_dim': [PCA()],
          'reduce_dim__n_components': [5,20,40,55],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]}
          ,
          {
          'scaler': scalers_to_test,
          'reduce_dim': [None],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]    
          }]

"""
pipe = Pipeline(steps=[('scaler', StandardScaler()),('reduce_dim', PCA()),('regressor', KMeans())],verbose=2)


scoring = {'accuracy': make_scorer(acc_score,greater_is_better=True,method='acc'),
           'prec': prec_score,
           'rec': rec_score,
           'f1_s': f1_s_score,
            'roc': roc_score}          

grid = GridSearchCV(pipe, cv=cvCount, scoring=scoring,param_grid= params,return_train_score=trainScore,refit=False,n_jobs=1, verbose=2)#

grid.fit(X_train,Y_train)

cvResult = pd.DataFrame(grid.cv_results_)

#rmtree(cachedir)

In [99]:
cvResult.sort_values(['rank_test_accuracy'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_reduce_dim,param_regressor__n_clusters,param_regressor__random_state,param_scaler,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_train_accuracy,split1_train_accuracy,split2_train_accuracy,split3_train_accuracy,split4_train_accuracy,mean_train_accuracy,std_train_accuracy,split0_test_prec,split1_test_prec,split2_test_prec,split3_test_prec,split4_test_prec,mean_test_prec,std_test_prec,rank_test_prec,split0_train_prec,split1_train_prec,split2_train_prec,split3_train_prec,split4_train_prec,mean_train_prec,std_train_prec,split0_test_rec,...,mean_test_rec,std_test_rec,rank_test_rec,split0_train_rec,split1_train_rec,split2_train_rec,split3_train_rec,split4_train_rec,mean_train_rec,std_train_rec,split0_test_f1_s,split1_test_f1_s,split2_test_f1_s,split3_test_f1_s,split4_test_f1_s,mean_test_f1_s,std_test_f1_s,rank_test_f1_s,split0_train_f1_s,split1_train_f1_s,split2_train_f1_s,split3_train_f1_s,split4_train_f1_s,mean_train_f1_s,std_train_f1_s,split0_test_roc,split1_test_roc,split2_test_roc,split3_test_roc,split4_test_roc,mean_test_roc,std_test_roc,rank_test_roc,split0_train_roc,split1_train_roc,split2_train_roc,split3_train_roc,split4_train_roc,mean_train_roc,std_train_roc
4,2.073998,0.035241,0.304873,0.011640,"PCA(copy=True, iterated_power='auto', n_compon...",2,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.425912,90.473544,90.481483,90.687889,90.481105,90.509987,0.091316,1,90.533983,90.516121,90.524060,90.461542,90.521177,90.511376,0.025589,0.991054,0.993658,0.993076,0.992639,0.992932,0.992672,0.000874,1,0.993080,0.992448,0.992575,0.992707,0.992635,0.992689,0.000213,0.801839,...,0.801996,0.001324,9,0.802103,0.801562,0.802683,0.801548,0.802184,0.802016,0.000426,0.886462,0.888445,0.886048,0.888051,0.887015,0.887204,0.000914,1,0.887433,0.886850,0.887586,0.886944,0.887305,0.887224,0.000283,0.897760,0.899396,0.897522,0.899166,0.898267,0.898422,0.000745,1,0.898621,0.898145,0.898721,0.898196,0.898504,0.898437,0.000229
5,2.053471,0.037828,0.303574,0.007960,"PCA(copy=True, iterated_power='auto', n_compon...",2,60000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.425912,90.473544,90.481483,90.687889,90.481105,90.509987,0.091316,1,90.533983,90.519098,90.524060,90.462534,90.522169,90.512369,0.025411,0.991054,0.993658,0.993076,0.992639,0.992932,0.992672,0.000874,1,0.993080,0.992449,0.992575,0.992707,0.992635,0.992689,0.000213,0.801839,...,0.801996,0.001324,9,0.802103,0.801627,0.802683,0.801569,0.802206,0.802037,0.000409,0.886462,0.888445,0.886048,0.888051,0.887015,0.887204,0.000914,1,0.887433,0.886889,0.887586,0.886957,0.887318,0.887237,0.000271,0.897760,0.899396,0.897522,0.899166,0.898267,0.898422,0.000745,1,0.898621,0.898177,0.898721,0.898207,0.898515,0.898448,0.000219
8,2.217137,0.030932,0.302927,0.005930,"PCA(copy=True, iterated_power='auto', n_compon...",2,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.374310,90.433851,90.437820,90.664073,90.473166,90.476644,0.098947,3,90.503220,90.489327,90.493297,90.428794,90.478506,90.478629,0.026145,0.990524,0.993343,0.992543,0.992424,0.993035,0.992374,0.000983,3,0.992892,0.992155,0.992310,0.992493,0.992263,0.992423,0.000259,0.801158,...,0.801519,0.001339,14,0.801591,0.801220,0.802236,0.801016,0.801566,0.801526,0.000415,0.885834,0.887959,0.885520,0.887754,0.886899,0.886793,0.000983,5,0.887045,0.886523,0.887207,0.886533,0.886778,0.886817,0.000273,0.897233,0.898989,0.897080,0.898920,0.898176,0.898080,0.000807,5,0.898300,0.897872,0.898404,0.897856,0.898065,0.898099,0.000222
9,2.167742,0.033027,0.304405,0.004308,"PCA(copy=True, iterated_power='auto', n_compon...",2,60000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': P

In [100]:
cc = ['param_reduce_dim','param_regressor__n_clusters','param_regressor__random_state','param_scaler','params','mean_test_accuracy','rank_test_accuracy','mean_test_prec','rank_test_prec','mean_test_rec','rank_test_rec','mean_test_f1_s','rank_test_f1_s','mean_test_roc','rank_test_roc']
s = cvResult[cc]
s.sort_values(['rank_test_accuracy'])

,param_reduce_dim,param_regressor__n_clusters,param_regressor__random_state,param_scaler,params,mean_test_accuracy,rank_test_accuracy,mean_test_prec,rank_test_prec,mean_test_rec,rank_test_rec,mean_test_f1_s,rank_test_f1_s,mean_test_roc,rank_test_roc
4,"PCA(copy=True, iterated_power='auto', n_compon...",2,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.509987,1,0.992672,1,0.801996,9,0.887204,1,0.898422,1
5,"PCA(copy=True, iterated_power='auto', n_compon...",2,60000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.509987,1,0.992672,1,0.801996,9,0.887204,1,0.898422,1
8,"PCA(copy=True, iterated_power='auto', n_compon...",2,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.476644,3,0.992374,3,0.801519,14,0.886793,5,0.898080,5
9,"PCA(copy=True, iterated_power='auto', n_compon...",2,60000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.476644,3,0.992374,3,0.801519,14,0.886793,5,0.898080,5
13,"PCA(copy=True, iterated_power='auto', n_compon...",2,60000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.475850,5,0.991939,5,0.801859,11,0.886827,3,0.898093,3
12,"PCA(copy=True, iterated_power='auto', n_compon...",2,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.475056,6,0.991918,6,0.801859,11,0.886818,4,0.898086,4
0,None,2,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': None, 'regressor__n_clusters': ...",90.429806,7,0.991118,7,0.801535,13,0.886301,7,0.897642,7
2,None,5,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': None, 'regressor__n_clusters': ...",86.356411,8,0.836505,9,0.894410,6,0.860729,8,0.865747,8
3,None,5,60000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': None, 'regressor__n_clusters': ...",85.350576,9,0.802101,10,0.918076,5,0.855055,13,0.857517,9
14,"PCA(copy=True, iterated_power='auto', n_compon...",5,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",84.975869,10,0.770322,13,0.964839,3,0.856670,9,0.857219,10


In [101]:


print(grid.cv_results_['param_reduce_dim'][4])
print(grid.cv_results_['params'][4])
 

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)
{'reduce_dim': PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False), 'regressor__n_clusters': 2, 'regressor__random_state': 0, 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True)}


In [102]:
print("Kmean 1\n\n")
loadData()
#df_train.columns
#print(df_train.groupby('attack_cat')['attack_cat'].count())
removeDuplicate()
#balanceClass()
#print(df_train.groupby('label')['label'].count())
#addOtherLabel()

splitData()


X_train, X_test = one_hot_encode(X_train,X_test)

X_train, X_test = scaling(X_train,X_test,StandardScaler())
X_train,X_test = runPCA(X_train,X_test,2)

kmean= KMeans(n_clusters=2,random_state=0)
kmean.fit(X_train)

Kmean 1


Duplicates train: 9
0
Duplicates test: 3
0
X_train shape : (125964, 122)
X_test shape : (22541, 122)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [103]:
predictedTrain = kmean.predict(X_train)
predictedTest= kmean.predict(X_test)
predictedTrain = 1 - predictedTrain 
predictedTest = 1 - predictedTest 

In [104]:

print("Train Accuracy : ",accuracy_score(Y_train,predictedTrain)*100)
print("Train set precision : {:.4f}".format(precision_score(Y_train, predictedTrain)))
print("Train set recall    : {:.4f}".format(recall_score(Y_train, predictedTrain)))
print("Train set F1-score  : {:.4f}".format(f1_score(Y_train, predictedTrain)))
print("Test set roc        : {:.4f}".format(roc_auc_score(Y_train, predictedTrain)))

print("Test Accuracy : ",accuracy_score(Y_test,predictedTest)*100)
print("Test set precision : {:.4f}".format(precision_score(Y_test, predictedTest)))
print("Test set recall    : {:.4f}".format(recall_score(Y_test, predictedTest)))
print("Test set F1-score  : {:.4f}".format(f1_score(Y_test, predictedTest)))
print("Test set roc       : {:.4f}".format(roc_auc_score(Y_test, predictedTest)))


Train Accuracy :  90.50919310279127
Train set precision : 0.9927
Train set recall    : 0.8020
Train set F1-score  : 0.8872
Test set roc        : 0.8984
Test Accuracy :  73.75005545450513
Test set precision : 0.9847
Test set recall    : 0.5473
Test set F1-score  : 0.7036
Test set roc       : 0.7680


In [ ]:
#.......................................................................with debug and useful parameter(Kmeans)
tmean=None
tvar=None

trainScore = True
cvCount = 5

last=1 #test=0
i_index=-1
myScore=[]
iCount=0
fitNum=0


def acc_score(actual,prediction,method):
  global last
  global i_index
  global myScore
  global iCount
  global fitNum
 

  m=[[],[],[],[],[]]
  cls = -1
  
  
  def oneVsAll(num):
    if (num ==cls):
      return 0
    else:
      return 1

  if (last == 0 and trainScore):
     #trainset
    print("in train")
    
    myScore.clear()
    cls = i_index
    #print("Index--------",i_index)
    oneVtest =np.array(list(map(oneVsAll, prediction)))
    myScore.append(accuracy_score(actual,oneVtest)*100)
    myScore.append(precision_score(actual,oneVtest))
    myScore.append(recall_score(actual,oneVtest))
    myScore.append(f1_score(actual,oneVtest))
    myScore.append(roc_auc_score(actual,oneVtest))

    last = 1
    
  else:
   #testset
    print("in test")
    lena = len(np.unique(actual))
    lenp = len(np.unique(prediction))
    if (lena>lenp):
      n = lena
    else:
      n= lenp
    if (iCount%cvCount == 0):
      
      fitNum =fitNum + 1
      print("-----------------------------------------------------------",fitNum,"--------------------------------------------------------")
      
       
    
    myScore.clear()
     """pp = pd.DataFrame(actual)
    ppp = pd.DataFrame(prediction)
    print(len(np.unique(actual)))
    print(len(np.unique(prediction)))
    print("actual : ",pp.groupby('class')['class'].count())
    if ((len(np.unique(actual))<2) or (len(np.unique(prediction))<2)):
      print("############################################################################################################################")
    print("predicted : ",ppp.groupby(0)[0].count())"""

    for i in range(0, n):
      cls=i
      oneVall=np.array(list(map(oneVsAll, prediction)))
      #print(len(np.unique(actual)))
      #pppp = pd.DataFrame(oneVall)
      #print("onevs : ",pppp.groupby(0)[0].count())
      #print(len(np.unique(oneVall)))
      #print("accurcy---",i,"  ",accuracy_score(actual,oneVall)*100)
      m[0].append(accuracy_score(actual,oneVall)*100)
      m[1].append(precision_score(actual,oneVall))
      m[2].append(recall_score(actual,oneVall))
      m[3].append(f1_score(actual,oneVall))
      m[4].append(roc_auc_score(actual,oneVall))
      
    
    i_index = m[0].index(max(m[0]))  
    #print("Index--------",i_index)
    myScore.append(m[0][i_index])
    myScore.append(m[1][i_index])
    myScore.append(m[2][i_index])
    myScore.append(m[3][i_index])
    myScore.append(m[4][i_index])
    print("accurcy max---",myScore[0])
    print("prec max---",myScore[1])
    print("rec max---",myScore[2])
    print("f1 max---",myScore[3])
    print("roc max---",myScore[4])

    last = 0
    #print("accurcy max---",myScore[0])
  return myScore[0]
  


b=0  

def prec_score(estimator,x,y):
  global b
  global tvar
  global tmean
  if (b == 0):
    b=1
    print(np.shape(x))
    #print("step 2: ",x)
    #tmean = estimator['scaler'].mean_
    #tvar = estimator['scaler'].var_
    tvar = estimator['scaler']
    tmean = estimator.get_params()
    print(np.shape(estimator['scaler'].mean_))
  return myScore[1]


def rec_score(estimator,x,y):
  return myScore[2]

def f1_s_score(estimator,x,y):
  return myScore[3]

def roc_score(estimator,x,y):
  return myScore[4]




loadData()

#print(df_train.groupby('label')['label'].count())
#addOtherLabel()
removeDuplicate()
#balanceClass()
splitData()

X_train, X_test = one_hot_encode(X_train,X_test)



scalers_to_test = [StandardScaler(), MinMaxScaler(),None]
 
params = {'scaler': scalers_to_test,
          'reduce_dim': [None,PCA(n_components=5),PCA(n_components=30),PCA(n_components=55)],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,50000,100000]}
          
"""
....................................or (both result same)
params = [{'scaler': scalers_to_test,
          'reduce_dim': [PCA()],
          'reduce_dim__n_components': [5,20,40,55],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]}
          ,
          {
          'scaler': scalers_to_test,
          'reduce_dim': [None],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]    
          }]

"""
pipe = Pipeline(steps=[('scaler', StandardScaler()),('reduce_dim', PCA()),('regressor', KMeans())],verbose=2)


scoring = {'accuracy': make_scorer(acc_score,greater_is_better=True,method='acc'),
           'prec': prec_score,
           'rec': rec_score,
           'f1_s': f1_s_score,
            'roc': roc_score}          

#cv= [(slice(None), slice(None))]# for non crossvalidation
#cv1 = ShuffleSplit(test_size=0.20, n_splits=1, random_state=0)#only use 1 cvset as 0.2 of training set

grid = GridSearchCV(pipe, cv=cvCount, scoring=scoring,param_grid= params,return_train_score=trainScore,refit=False,n_jobs=1, verbose=2)#

grid.fit(X_train,Y_train)

cvResult = pd.DataFrame(grid.cv_results_)

cvResult
#rmtree(cachedir)


cc = ['param_reduce_dim','param_regressor__n_clusters','param_regressor__random_state','param_scaler','params','mean_test_accuracy','rank_test_accuracy','mean_test_prec','rank_test_prec','mean_test_rec','rank_test_rec','mean_test_f1_s','rank_test_f1_s','mean_test_roc','rank_test_roc']
s = cvResult[cc]
s.sort_values(['rank_test_accuracy'])
print(s.sort_values(['rank_test_accuracy']))
tvar.mean_
tmean

In [ ]:
#.......................................................................GMM.............
trainScore = True
cvCount = 5

last=1 #test=0
i_index=-1
myScore=[]
iCount=0
fitNum=0


def acc_score(actual,prediction,method):
  global last
  global i_index
  global myScore
  global iCount
  global fitNum
 

  m=[[],[],[],[],[]]
  cls = -1
  
  
  def oneVsAll(num):
    if (num ==cls):
      return 0
    else:
      return 1

  if (last == 0 and trainScore):
     #trainset
    print("in train")
    
    myScore.clear()
    cls = i_index
    #print("Index--------",i_index)
    oneVtest =np.array(list(map(oneVsAll, prediction)))
    myScore.append(accuracy_score(actual,oneVtest)*100)
    myScore.append(precision_score(actual,oneVtest))
    myScore.append(recall_score(actual,oneVtest))
    myScore.append(f1_score(actual,oneVtest))
    myScore.append(roc_auc_score(actual,oneVtest))

    last = 1
    
  else:
   #testset
    print("in test")
    lena = len(np.unique(actual))
    lenp = len(np.unique(prediction))
    if (lena>lenp):
      n = lena
    else:
      n= lenp
    if (iCount%cvCount == 0):
      
      fitNum =fitNum + 1
      print("-----------------------------------------------------------",fitNum,"--------------------------------------------------------")

    myScore.clear()
    
    for i in range(0, n):
      cls=i
      oneVall=np.array(list(map(oneVsAll, prediction)))
      #print(len(np.unique(actual)))
      #pppp = pd.DataFrame(oneVall)
      #print("onevs : ",pppp.groupby(0)[0].count())
      #print(len(np.unique(oneVall)))
      #print("accurcy---",i,"  ",accuracy_score(actual,oneVall)*100)
      m[0].append(accuracy_score(actual,oneVall)*100)
      m[1].append(precision_score(actual,oneVall))
      m[2].append(recall_score(actual,oneVall))
      m[3].append(f1_score(actual,oneVall))
      m[4].append(roc_auc_score(actual,oneVall))
      
    
    i_index = m[0].index(max(m[0]))  
    #print("Index--------",i_index)
    myScore.append(m[0][i_index])
    myScore.append(m[1][i_index])
    myScore.append(m[2][i_index])
    myScore.append(m[3][i_index])
    myScore.append(m[4][i_index])

    last = 0
    #print("accurcy max---",myScore[0])
  return myScore[0]
  


def prec_score(estimator,x,y):
  return myScore[1]

def rec_score(estimator,x,y):
  return myScore[2]

def f1_s_score(estimator,x,y):
  return myScore[3]

def roc_score(estimator,x,y):
  return myScore[4]




loadData()

#print(df_train.groupby('label')['label'].count())
#addOtherLabel()
removeDuplicate()
#balanceClass()
splitData()

X_train, X_test = one_hot_encode(X_train,X_test)


cv_types = [ 'tied', 'diag', 'full']
scalers_to_test = [StandardScaler(), MinMaxScaler(),None]
 
params = {'scaler': scalers_to_test,
          'reduce_dim': [None,PCA(n_components=5),PCA(n_components=30),PCA(n_components=55)],
          'regressor__n_components': [2,5],
          'regressor__covariance_type': cv_types,
          'regressor__random_state':[0,2000,50000,100000]}
          
"""
....................................or (both result same in reduce__dim)
params = [{'scaler': scalers_to_test,
          'reduce_dim': [PCA()],
          'reduce_dim__n_components': [5,20,40,55],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]}
          ,
          {
          'scaler': scalers_to_test,
          'reduce_dim': [None],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]    
          }]

"""
pipe = Pipeline(steps=[('scaler', StandardScaler()),('reduce_dim', PCA()),('regressor', GaussianMixture())],verbose=2)


scoring = {'accuracy': make_scorer(acc_score,greater_is_better=True,method='acc'),
           'prec': prec_score,
           'rec': rec_score,
           'f1_s': f1_s_score,
            'roc': roc_score}          



grid = GridSearchCV(pipe, cv=cvCount, scoring=scoring,param_grid= params,return_train_score=trainScore,refit='accuracy',n_jobs=1, verbose=2)#

grid.fit(X_train,Y_train)

cvResult = pd.DataFrame(grid.cv_results_)

#rmtree(cachedir)

In [38]:
cvResult.sort_values(['rank_test_accuracy'])


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_reduce_dim,param_regressor__covariance_type,param_regressor__n_components,param_regressor__random_state,param_scaler,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_train_accuracy,split1_train_accuracy,split2_train_accuracy,split3_train_accuracy,split4_train_accuracy,mean_train_accuracy,std_train_accuracy,split0_test_prec,split1_test_prec,split2_test_prec,split3_test_prec,split4_test_prec,mean_test_prec,std_test_prec,rank_test_prec,split0_train_prec,split1_train_prec,split2_train_prec,split3_train_prec,split4_train_prec,mean_train_prec,std_train_prec,...,mean_test_rec,std_test_rec,rank_test_rec,split0_train_rec,split1_train_rec,split2_train_rec,split3_train_rec,split4_train_rec,mean_train_rec,std_train_rec,split0_test_f1_s,split1_test_f1_s,split2_test_f1_s,split3_test_f1_s,split4_test_f1_s,mean_test_f1_s,std_test_f1_s,rank_test_f1_s,split0_train_f1_s,split1_train_f1_s,split2_train_f1_s,split3_train_f1_s,split4_train_f1_s,mean_train_f1_s,std_train_f1_s,split0_test_roc,split1_test_roc,split2_test_roc,split3_test_roc,split4_test_roc,mean_test_roc,std_test_roc,rank_test_roc,split0_train_roc,split1_train_roc,split2_train_roc,split3_train_roc,split4_train_roc,mean_train_roc,std_train_roc
97,6.035191,0.154535,0.648587,0.038509,"PCA(copy=True, iterated_power='auto', n_compon...",full,2,100000,"MinMaxScaler(copy=True, feature_range=(0, 1))","{'reduce_dim': PCA(copy=True, iterated_power='...",90.362402,90.390188,90.477514,90.568809,90.449349,90.449652,0.072282,1,90.463526,90.458564,90.452610,90.415893,90.454690,90.449057,0.016994,0.995110,0.996854,0.996887,0.996120,0.996280,0.996250,0.000646,28,0.996298,0.996070,0.996147,0.996335,0.996244,0.996219,0.000098,...,0.797781,0.000809,67,0.797965,0.797368,0.798233,0.797615,0.797811,0.797798,0.000295,0.885201,0.887033,0.885604,0.886098,0.886252,0.886037,0.000621,1,0.886170,0.885711,0.886275,0.885969,0.886054,0.886036,0.000193,0.896868,0.898376,0.897264,0.897659,0.897754,0.897584,0.000505,1,0.897693,0.897324,0.897769,0.897523,0.897598,0.897581,0.000153
54,6.496181,0.158218,0.645157,0.026358,"PCA(copy=True, iterated_power='auto', n_compon...",tied,2,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.251260,90.298893,90.290954,90.485452,90.358050,90.336922,0.081737,2,90.367268,90.355360,90.353375,90.289865,90.327670,90.338708,0.027625,0.986689,0.989234,0.987714,0.988097,0.988669,0.988081,0.000866,42,0.988642,0.988000,0.988203,0.987822,0.987808,0.988095,0.000309,...,0.802029,0.001097,64,0.802146,0.801734,0.802598,0.801867,0.801971,0.802063,0.000299,0.884608,0.886621,0.884065,0.885808,0.885828,0.885386,0.000922,2,0.885683,0.885174,0.885782,0.885184,0.885242,0.885413,0.000264,0.896113,0.897737,0.895767,0.897241,0.897168,0.896805,0.000741,2,0.897065,0.896657,0.897117,0.896606,0.896681,0.896825,0.000219
79,5.810494,0.090230,0.613563,0.028602,"PCA(copy=True, iterated_power='auto', n_compon...",diag,2,100000,"MinMaxScaler(copy=True, feature_range=(0, 1))","{'reduce_dim': PCA(copy=True, iterated_power='...",90.255230,90.223475,90.318739,90.477514,90.306446,90.316281,0.087680,3,90.331544,90.339483,90.318643,90.280934,90.319732,90.318067,0.020111,0.992262,0.993306,0.993997,0.993869,0.993842,0.993455,0.000642,35,0.993725,0.993463,0.993370,0.993481,0.993512,0.993510,0.000117,...,0.797168,0.000936,68,0.797197,0.796897,0.797594,0.797019,0.797107,0.797163,0.000237,0.884072,0.885212,0.883774,0.885100,0.884604,0.884553,0.000561,4,0.884678,0.884389,0.884782,0.884472,0.884538,0.884572,0.000141,0.895864,0.896761,0.895710,0.896801,0.896345,0.896296,0.000448,4,0.896409,0.896182,0.896474,0.896221,0.896290,0.896315,0.000111
115,10.721738,0.166108,0.786919,0.039806,"PCA(copy=True, iterated_power='auto', n_compon...",tied,2,100000,"MinMaxScaler(copy=True, feature_range=(0, 1))"

In [39]:
cc = ['param_reduce_dim','param_regressor__covariance_type', 'param_regressor__n_components','param_regressor__random_state','param_scaler','params','mean_test_accuracy','rank_test_accuracy','mean_test_prec','rank_test_prec','mean_test_rec','rank_test_rec','mean_test_f1_s','rank_test_f1_s','mean_test_roc','rank_test_roc']
s = cvResult[cc]
s.sort_values(['rank_test_accuracy'])

,param_reduce_dim,param_regressor__covariance_type,param_regressor__n_components,param_regressor__random_state,param_scaler,params,mean_test_accuracy,rank_test_accuracy,mean_test_prec,rank_test_prec,mean_test_rec,rank_test_rec,mean_test_f1_s,rank_test_f1_s,mean_test_roc,rank_test_roc
97,"PCA(copy=True, iterated_power='auto', n_compon...",full,2,100000,"MinMaxScaler(copy=True, feature_range=(0, 1))","{'reduce_dim': PCA(copy=True, iterated_power='...",90.449652,1,0.996250,28,0.797781,67,0.886037,1,0.897584,1
54,"PCA(copy=True, iterated_power='auto', n_compon...",tied,2,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.336922,2,0.988081,42,0.802029,64,0.885386,2,0.896805,2
79,"PCA(copy=True, iterated_power='auto', n_compon...",diag,2,100000,"MinMaxScaler(copy=True, feature_range=(0, 1))","{'reduce_dim': PCA(copy=True, iterated_power='...",90.316281,3,0.993455,35,0.797168,68,0.884553,4,0.896296,4
115,"PCA(copy=True, iterated_power='auto', n_compon...",tied,2,100000,"MinMaxScaler(copy=True, feature_range=(0, 1))","{'reduce_dim': PCA(copy=True, iterated_power='...",90.306755,4,0.990569,39,0.799316,66,0.884724,3,0.896347,3
90,"PCA(copy=True, iterated_power='auto', n_compon...",full,2,0,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",90.266267,5,0.996910,23,0.793293,70,0.883521,5,0.895578,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,"PCA(copy=True, iterated_power='auto', n_compon...",full,5,0,passthrough,"{'reduce_dim': PCA(copy=True, iterated_power='...",NaN,212,NaN,212,NaN,212,NaN,212,NaN,212
98,"PCA(copy=True, iterated_power='auto', n_compon...",full,2,100000,passthrough,"{'reduce_dim': PCA(copy=True, iterated_power='...",NaN,213,NaN,213,NaN,213,NaN,213,NaN,213
92,"PCA(copy=True, iterated_power='auto', n_compon...",full,2,0,passthrough,"{'reduce_dim': PCA(copy=True, iterated_power='...",NaN,214,NaN,214,NaN,214,NaN,214,NaN,214
161,"PCA(copy=True, iterated_power='auto', n_compon...",full,5,100000,passthrough,"{'reduce_dim': PCA(copy=True, iterated_power='...",NaN,215,NaN,215,NaN,215,NaN,215,NaN,215


In [46]:
nu = 54

print(grid.cv_results_['param_reduce_dim'][nu])
print(grid.cv_results_['params'][nu])

PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)
{'reduce_dim': PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False), 'regressor__covariance_type': 'tied', 'regressor__n_components': 2, 'regressor__random_state': 0, 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True)}


In [50]:
print("GMM \n\n")
loadData()
#df_train.columns
#print(df_train.groupby('attack_cat')['attack_cat'].count())

#print(df_train.groupby('label')['label'].count())
#addOtherLabel()
removeDuplicate()
#balanceClass()
splitData()


X_train, X_test = one_hot_encode(X_train,X_test)

X_train, X_test = scaling(X_train,X_test,StandardScaler())
X_train,X_test = runPCA(X_train,X_test,5)

gmm= GaussianMixture(n_components=2,random_state=0,covariance_type='tied')
gmm.fit(X_train)

GMM 


Duplicates train: 9
0
Duplicates test: 3
0
X_train shape : (125964, 122)
X_test shape : (22541, 122)


GaussianMixture(covariance_type='tied', init_params='kmeans', max_iter=100,
                means_init=None, n_components=2, n_init=1, precisions_init=None,
                random_state=0, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [51]:
predictedTrain = gmm.predict(X_train)
predictedTest= gmm.predict(X_test)

In [52]:
oneVsAllScore(2,predictedTrain, predictedTest)

0    0.09659902829379823
1    0.9034009717062018
Accuracy Train:  90.34009717062018
Accuracy Test:  73.82990994188367
Test set precision : 0.9875
Test set recall    : 0.5472
Test set F1-score  : 0.7041
Test set Roc       : 0.7690
True Positives  : 0.55
True Negatives  : 0.99
False Positives : 0.01
False Negatives : 0.45
